Google Colab上執行  
使用Lena，放在google colab的sample_data目錄下後再進行存取  
SourceCode@Google Colab: https://colab.research.google.com/drive/1ly5NPMbKH5osuqqwiefAwIh_mhNcEFin

In [0]:
# _*_ coding: utf-8 _*_
# _*_ coding: cp950 _*_

# author: Chi-Hsu Chen (css920@gmail.com)

import numpy as np
import cv2
import time
from google.colab.patches import cv2_imshow

from keras import utils
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model,Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.layers import Input,GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.layers import BatchNormalization,Activation
from keras import layers
from keras import backend as K

Inception V1網路架構
![alt text](https://github.com/ChiHsuChen/DL-CVMarathon/blob/master/homework/Inception%E6%9E%B6%E6%A7%8B.png?raw=true)

# 建構Inception V1&V3 Block

In [0]:
# 建構含有batch normalization的Conv2D
def Conv2D_BatchNormalization(block,filters,kernel_size,padding='same',strides=(1,1),normalizer=True,activation='relu',name=None):
  if name is not None:
    conv_name=name+'_conv'
    bn_name=name+'_bn'
    activation_name=name+'_act'
  else:
    conv_name=None
    bn_name=None
    activation_name=None
  
  if K.image_data_format() == 'channels_first':
    bn_axis = 1
  else:
    bn_axis = 3

  # 非sequential式使用函數式建構
  block=Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding=padding,
               use_bias=False,
               name=conv_name)(block)
  
  if normalizer==True:
    block=BatchNormalization(axis=bn_axis,scale=False,name=bn_name)(block)
  
  if activation==True:
    block=Activation(activation=activation,name=act_name)
  
  return block

In [0]:
# 參考: https://github.com/keras-team/keras-applications/blob/master/keras_applications/inception_v3.py
def InceptionV1_Block(block,specs,channel_axis,name):
  (br0, br1, br2, br3)=specs  # ((64,), (96,128), (16,32), (32,))
  
  # Branch 0 >> 1*1 Conv
  branch_0=Conv2D_BatchNormalization(block,br0[0],(1,1),name=name+'branch_0')

  # Branch 1 >> 1*1 Conv -> 3*3 Conv
  branch_1=Conv2D_BatchNormalization(block,br1[0],(1,1),name=name+'branch_1')
  branch_1=Conv2D_BatchNormalization(branch_1,br1[1],(3,3),name=name+'branch_1_1')

  # Branch 2 >> 1*1 Conv -> 5*5 Conv
  branch_2=Conv2D_BatchNormalization(block,br2[0],(1,1),name=name+'branch_2')
  branch_2=Conv2D_BatchNormalization(branch_2,br2[1],(5,5),name=name+'branch_2_1')

  # Branch 3 >> 3*3 MaxPooling -> 1*1 Conv
  branch_3=MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name=name+'branch_3')(block)
  branch_3=Conv2D_BatchNormalization(branch_3,br3[0],(3,3),name=name+'branch_3_1')
  
  # 合併
  block=layers.concatenate([branch_0,branch_1,branch_2,branch_3],
                           axis=channel_axis,
                           name=name+'_concate')
  
  return block


In [38]:
# 測試
img_input=Input(shape=(224,224,1))
x=InceptionV1_Block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
print(x)

Tensor("Block_1_concate_5/concat:0", shape=(None, 224, 224, 256), dtype=float32)


In [0]:
# 參考: https://github.com/keras-team/keras-applications/blob/master/keras_applications/inception_v3.py
# 將 InceptionV1_block中n*n卷積改為1 x n+n x 1
def InceptionV3_block(x, specs,channel_axis, name):
  (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
  branch_0 = Conv2D_BatchNormalization(x, br0[0], (1, 1), name=name+"_Branch_0")
    
  # 'branch 1
  branch_1 = Conv2D_BatchNormalization(x, br1[0], (1, 1), name=name+"_Branch_1")
  branch_1 = Conv2D_BatchNormalization(branch_1, br1[1], (1, 3), name=name+"_Branch_1_1")
  branch_1 = Conv2D_BatchNormalization(branch_1, br1[1], (3, 1), name=name+"_Branch_1_2")

  # '''Branch_2'''
  branch_2 = Conv2D_BatchNormalization(x, br2[0], (1, 1), name=name+"_Branch_2")
  branch_2 = Conv2D_BatchNormalization(branch_1, br2[1], (1, 5), name=name+"_Branch_2_1")
  branch_2 = Conv2D_BatchNormalization(branch_1, br2[1], (5, 1), name=name+"_Branch_2_2")

  # '''Branch_3'''
  branch_3=MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name=name+'branch_3')(x)
  branch_3=Conv2D_BatchNormalization(branch_3,br3[0],(1,3),name=name+'branch_3_1')
  branch_3=Conv2D_BatchNormalization(branch_3,br3[0],(3,1),name=name+'branch_3_2')

  x = layers.concatenate(
      [branch_0, branch_1, branch_2, branch_3],
      axis=channel_axis,
      name=name+"_Concatenated")
  
  return x

In [49]:
img_input = Input(shape=(224,224,1))
x=InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
print(x)

Tensor("Block_1_Concatenated/concat:0", shape=(None, 224, 224, 256), dtype=float32)


# 額外練習
將VGG16  
Block_3中的Convolution全部改為InceptionV1_block  
Block_5中的Convolution全部改為InceptionV3_block  
並將所有Convolution改為Conv2d_bn

In [0]:
def VGG16_Inception(include_top=True,input_tensor=None, input_shape=(224,224,1), pooling='max',classes=1000):
 
    '''修改模型'''
    img_input = Input(shape=input_shape)

    x = Conv2D_BatchNormalization(img_input,64, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = Conv2D_BatchNormalization(x,64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D_BatchNormalization(x,128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2D_BatchNormalization(x,128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    #x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    #x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    #x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x=InceptionV1_Block(x, ((64,), (96,128), (16,32), (32,)), 3, 'block3_conv1')
    x=InceptionV1_Block(x, ((64,), (96,128), (16,32), (32,)), 3, 'block3_conv2')
    x=InceptionV1_Block(x, ((64,), (96,128), (16,32), (32,)), 3, 'block3_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D_BatchNormalization(x,512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2D_BatchNormalization(x,512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2D_BatchNormalization(x,512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    #x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    #x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    #x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x=InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'block5_conv1')
    x=InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'block5_conv2')
    x=InceptionV3_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'block5_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')
   
    return model

In [63]:
model = VGG16_Inception(include_top=False)
model.summary()

Model: "vgg16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_21[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 224, 224, 64) 192         block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
block1_conv2_conv (Conv2D)      (None, 224, 224, 64) 36864       block1_conv1_bn[0][0]            
______________________________________________________________________________________________